---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. Use this dataset to answer the questions below.

In [2]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

# Summer  Gold  Silver  Bronze  Total  # Winter  Gold.1  \
Afghanistan        13     0       0       2      2         0       0   
Algeria            12     5       2       8     15         3       0   
Argentina          23    18      24      28     70        18       0   
Armenia             5     1       2       9     12         6       0   
Australasia         2     3       4       5     12         0       0   

             Silver.1  Bronze.1  Total.1  # Games  Gold.2  Silver.2  Bronze.2  \
Afghanistan         0         0        0       13       0         0         2   
Algeria             0         0        0       15       5         2         8   
Argentina           0         0        0       41      18        24        28   
Armenia             0         0        0       11       1         2         9   
Australasia         0         0        0        2       3         4         5   

             Combined total   ID  
Afghanistan               2  AFG  
Algeria                  15  ALG  
Argentina                70  ARG  
Armenia                  12  ARM  
Australasia              12  ANZ

### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [3]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero()

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [4]:
def answer_one():
    return df[df['Gold'] == df['Gold'].max()].index[0]

answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [5]:
def answer_two():
    # create new row containing abs value of Gold and Gold.1 differences
    df['diff'] = abs(df['Gold'] - df['Gold.1'])
    # index / name from the element with the largest difference
    return df[df['diff'] == df['diff'].max()].index[0]

answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer and winter gold medal counts relative to their total gold medal count? Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [6]:
def answer_three():
    # create sub dataframe of only elements with greater than 0 Gold and Gold.1
    sub_df = df[(df['Gold'] > 0) & (df['Gold.1'] > 0)].copy()
    # calculate difference between Gold medals relative to total metal count
    sub_df['rel'] = sub_df['diff'] / sub_df['Gold.2']
    return sub_df[sub_df['rel'] == max(sub_df['rel'])].index[0]

answer_three()

'Bulgaria'

### Question 4
Write a function to update the dataframe to include a new column called "Points" which is a weighted value where each gold medal counts for 3 points, silver medals for 2 points, and bronze mdeals for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [13]:
def answer_four():
    df['Points'] = df['Gold.2'] * 3 + df['Silver.2'] * 2 + df['Bronze.2'] * 1
    return df['Points']

answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [14]:
census_df = pd.read_csv('census.csv')
census_df.head()

SUMLEV  REGION  DIVISION  STATE  COUNTY   STNAME         CTYNAME  \
0      40       3         6      1       0  Alabama         Alabama   
1      50       3         6      1       1  Alabama  Autauga County   
2      50       3         6      1       3  Alabama  Baldwin County   
3      50       3         6      1       5  Alabama  Barbour County   
4      50       3         6      1       7  Alabama     Bibb County   

   CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010     ...       \
0        4779736            4780127          4785161     ...        
1          54571              54571            54660     ...        
2         182265             182265           183193     ...        
3          27457              27457            27341     ...        
4          22915              22919            22861     ...        

   RDOMESTICMIG2011  RDOMESTICMIG2012  RDOMESTICMIG2013  RDOMESTICMIG2014  \
0          0.002295         -0.193196          0.381066          0.582002   
1          7.242091         -2.915927         -3.012349          2.265971   
2         14.832960         17.647293         21.845705         19.243287   
3         -4.728132         -2.500690         -7.056824         -3.904217   
4         -5.527043         -5.068871         -6.201001         -0.177537   

   RDOMESTICMIG2015  RNETMIG2011  RNETMIG2012  RNETMIG2013  RNETMIG2014  \
0         -0.467369     1.030015     0.826644     1.383282     1.724718   
1         -2.530799     7.606016    -2.626146    -2.722002     2.592270   
2         17.197872    15.844176    18.559627    22.727626    20.317142   
3        -10.543299    -4.874741    -2.758113    -7.167664    -3.978583   
4          0.177258    -5.088389    -4.363636    -5.403729     0.754533   

   RNETMIG2015  
0     0.712594  
1    -2.187333  
2    18.293499  
3   -10.543299  
4     1.107861  

[5 rows x 100 columns]

In [15]:
def answer_five():
    return census_df.groupby('STNAME')['COUNTY'].count().idxmax()

answer_five()

'Texas'

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)?

*This function should return a list of string values.*

In [79]:
def answer_six():
    df1 = pd.DataFrame(census_df.where(census_df['SUMLEV'] == 50).groupby(['STNAME'])['POPESTIMATE2015'].nlargest(3))
    df1 = df1.reset_index()

    return list(df1.groupby(['STNAME']).sum()['POPESTIMATE2015'].nlargest(3).index)

answer_six()

def answer_six_wrong():
    d = census_df.where(census_df['SUMLEV'] == 50)
    # returns California, Illinoise, Texas
    return d.groupby(['STNAME', 'COUNTY']).POPESTIMATE2015.max().nlargest(3).reset_index()['STNAME'].tolist()



#d = census_df.where(census_df['SUMLEV'] == 50)
#d.groupby(['STNAME', 'COUNTY']).POPESTIMATE2015.max().nlargest(3)#.reset_index()#['STNAME'].tolist()
#df2 = census_df.groupby('STNAME')['POPESTIMATE2015'].nlargest(3).reset_index().groupby('STNAME').sum() 
#df2.sort_values().nlargest(3, columns='POPESTIMATE2015').index.tolist()
#df2.nlargest(3, columns='POPESTIMATE2015').sort().index.tolist()
#df2.sort(columns = 'POPESTIMATE2015', ascending=False)

#d = census_df.where(census_df['SUMLEV'] == 50)
#d.groupby(['STNAME', 'COUNTY']).POPESTIMATE2015.max().nlargest(3).reset_index()

#d.groupby(['STNAME', 'COUNTY']).POPESTIMATE2015.max().nlargest(3)

#census_df[census_df['SUMLEV']==50].sort('POPESTIMATE2015', ascending = False).POPESTIMATE2015

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest change in population within the five year period (hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all five columns)?

*This function should return a single string value.*

In [316]:
def answer_seven():
    d = census_df[census_df.SUMLEV==50].copy()
    d['max'] = d[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].max(axis=1)
    d['min'] = d[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].min(axis=1)
    d['diff'] = d['max'] - d['min']

    return d[d['diff'] == d['diff'].max()].iloc[0]['CTYNAME']

answer_seven()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [336]:
def answer_eight():

    return census_df[((census_df['REGION'] == 1) | (census_df['REGION'] == 2)) & (census_df['CTYNAME'].str.startswith("Washington")) & (cen_df['POPESTIMATE2015'] > cen_df['POPESTIMATE2014'])].loc[:,['STNAME','CTYNAME']]

answer_eight()

STNAME            CTYNAME
1211         Maine  Washington County
1918      New York  Washington County
2345  Pennsylvania  Washington County
2355  Rhode Island  Washington County
2863       Vermont  Washington County